<a href="https://colab.research.google.com/github/DariaPokh/Data_analysis/blob/main/%D0%9F%D0%BE%D1%85_%D0%94_%D0%A1_%D0%A0%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%B4%D0%BE%D0%BA%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D0%B5%D0%B9_API_VK_%D0%A1%D0%B1%D0%BE%D1%80_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B8%D0%B7_VK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Задание 1. Работа с документацией API VK**

С помощью методов API ВКонтакте получите список высших учебных заведений и названия их факультетов в городе Томск.
Результат должен быть записан в файл JSON в следующем формате:

In [ ]:
import requests
import json

ACCESS_TOKEN = "132e7bea57998e1874459eabbbabd9b834dade417af1adf7aab6bae88dfdd7b6f67a53ada67de5a17aebf"
COUNTRY_CODE = "RU"

URL_UNIVERSITIES = "https://api.vk.com/method/database.getUniversities"
URL_FACULTIES = "https://api.vk.com/method/database.getFaculties"

COUNTRY_ID = 1
CITY_ID = 144

const_params = {"access_token": ACCESS_TOKEN, "v": "5.131"}

try:
    params = {
      **const_params,
      "country_id": COUNTRY_ID,
      "city_id": CITY_ID}
      
    resp = requests.get(URL_UNIVERSITIES, params=params).json()
    
    if resp.get("error"): raise("Get University error!")

    universities = resp['response']['items']
    if len(universities) < 1: raise("No Universities was found!")

    results = {
      "cities": [
    {   
        "id": CITY_ID,
        "name": "Томск",
        "universities": []
     }]}

    for university in universities:
      university_id = university['id']
      university_name = university['title']

      params = {**const_params, "university_id": university_id}
       
      resp = requests.get(URL_FACULTIES, params=params).json()
       
      if resp.get("error"): raise("Get faculties error!")
      faculties = resp['response']['items']
       
      if len(faculties) < 1: raise("No faculties were found!")
       
      faculty_titles = []

      for faculty in faculties:
        faculty_titles.append(faculty['title'])
        
      data = {
          "id": university_id, 
          "name": university_name,
          "faculties": faculty_titles
          }
      results['cities'][0]['universities'].append(data)

except Exception as ex:
    print(f"{ex}")

finally:
  with open("Universities.json", "w") as created_file:
      json.dump(results, created_file, ensure_ascii=False)


exceptions must derive from BaseException


# **Задание 2. Сбор данных из социальной сети VK**

С помощью методов API ВКонтакте получите 1000 подписчиков группы "Лентач", отсортирванных по дате регистрации.
Вам необходимо собрать следующие данные в CSV файл: пол, название город, семейное положение (ФИО партнера не указывать).

In [ ]:
import requests
import csv



ACCESS_TOKEN = "132e7bea57998e1874459eabbbabd9b834dade417af1adf7aab6bae88dfdd7b6f67a53ada67de5a17aebf"
COUNTRY_CODE = "RU"

URL_MEMBERS = "https://api.vk.com/method/groups.getMembers"

const_params = {"access_token": ACCESS_TOKEN, "v": "5.131"}
list_members = []
fieldnames = ['Пол', 'Город', 'Семейное положение']

try:
    params = {
      **const_params,
      'group_id': 'lentach',
      'sort': 'id_asc',
      'count': 1000,
      'fields': 'relation, sex, city'}
    
    resp = requests.get(URL_MEMBERS, params=params).json()
    if resp.get("error"): raise("Get member error!")
    members = resp['response']['items']
    if len(members) < 1: raise("No member was found!")
    for member in members:
      sex_member = 'мужчина' if member['sex'] == 1 else 'женщина'
      try:
        member_city = member['city']['title']
      except Exception:
        member_city = 'не указан'
      try:
        member_relation = member['relation']
        if member_relation == 1:
          member_relation = 'не женат/не замужем'
        elif member_relation == 2:
          member_relation = 'есть друг/есть подруга'
        elif member_relation == 3:
          member_relation = 'помолвлен/помолвлена'   
        elif member_relation == 4:
          member_relation = ' женат/замужем'
        elif member_relation == 5:
          member_relation = 'всё сложно' 
        elif member_relation == 6:
          member_relation = 'в активном поиске'
        elif member_relation == 7:
          member_relation = 'влюблён/влюблена'   
        elif member_relation == 8:
          member_relation = 'в гражданском браке'
        elif member_relation == 0:
          member_relation = 'не указано'    
      except Exception:
        member_relation = 'не указан'

      data = {"Пол":sex_member, "Город": member_city, "Семейное положение": member_relation}
      list_members.append(data)

except Exception as ex:
    print(f"{ex}")
   
finally:
  filename = "members.csv"
  f = open(filename, "w")
  writer = csv.DictWriter(f, fieldnames=list_members[0].keys())
  writer.writeheader()
  for member in list_members:
    writer.writerow(member)

